# Validate your requirements.yaml file

You might have created your `requirements.yaml` file manually or using the requirements generator notebook and you have modified it. Use this notebook to validate that the file is correctly formatted. You can use this notebook to validate your `requirements.yaml` file before uploading it to your repository.

> **NOTE**: You do not need to run this notebook in the same environment as the one used to run the notebook you are uploading.

In [ ]:
# @markdown Validate your requirements.yaml file

import yaml
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display

required_keys = {"dependencies", "python_version", "description"}
alt_python_key = "python"

def validate_requirements(path: Path):
    if not path.exists():
        return False, [f"File not found: {path}"]
    try:
        with path.open("r", encoding="utf-8") as f:
            data = yaml.safe_load(f)
    except Exception as exc:
        return False, [f"YAML read error: {exc}"]
    if not isinstance(data, dict):
        return False, ["YAML root must be a mapping (key/value pairs)."]

    keys = set(data.keys())
    missing = required_keys - keys
    messages = []
    python_key = "python_version"
    if "python_version" not in data and alt_python_key in data:
        python_key = alt_python_key
        missing -= {"python_version"}
    if missing:
        messages.append("Missing required keys: " + ", ".join(sorted(missing)))

    deps = data.get("dependencies")
    if deps is None:
        messages.append("Key 'dependencies' is required.")
    elif not isinstance(deps, list):
        messages.append("'dependencies' must be a YAML list (lines prefixed with '- ').")
    else:
        for d in deps:
            if not isinstance(d, str):
                messages.append(f"{d} (is not a string)")
            elif not d.strip():
                messages.append(f"{d} (is an empty string)")
            elif ' - ' in d:
                messages.append(f"{d} (contains ' - ': possible indentation error in YAML)")

    if python_key not in data:
        messages.append("'python_version' is required.")
    elif not isinstance(data.get(python_key), str) or not data.get(python_key).strip():
        messages.append(f"'{python_key}' must be a non-empty string (for example: 3.10.13).")

    if "description" not in data or not isinstance(data.get("description"), str) or not data.get("description").strip():
        messages.append("'description' must be a non-empty string.")

    return len(messages) == 0, messages

path_input = widgets.Text(
    value="requirements.yaml",
    description="Path:",
    layout=widgets.Layout(width="99%"),
    style={"description_width": "initial"},
)
validate_btn = widgets.Button(
    description="Validate requirements.yaml",
    button_style="success",
    layout=widgets.Layout(width="99%"),
)
output_html = widgets.HTML()

def on_validate(_):
    path = Path(path_input.value).expanduser().resolve()
    ok, messages = validate_requirements(path)
    if ok:
        output_html.value = f"<b>✅ Validation passed for {path.name}.</b>"
    else:
        output_html.value = f"<b>⚠️ Issues found in the given {path.name}:</b><ul>"
        for msg in messages:
            output_html.value += f"<li>❌ {msg}</li>"
        output_html.value += "</ul>"

validate_btn.on_click(on_validate)
display(path_input, validate_btn, output_html)
